In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
from sqlalchemy import create_engine

In [2]:
# Census API Key
from config import api_key
c = Census(api_key, year=2018)

In [3]:
# Run Census Search to retrieve data on all zip codes (2018 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e",

census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E", "B19301_001E", "B17001_002E", "B25034_001E",
                         "B25034_002E", "B25034_003E", "B25034_004E", "B25034_005E", "B25034_006E", "B25034_007E",
                         "B25034_008E", "B25034_009E", "B25034_010E", "B25034_011E"),{'for': 'zip code tabulation area:*'})

In [4]:
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

In [5]:
# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "population",
                                      "B01002_001E": "median_age",
                                      "B19013_001E": "median_household_income",
                                      "B19301_001E": "per_capita_income",
                                      "B17001_002E": "poverty_count",
                                      "B25034_001E": "year_structure_built_total",
                                      "B25034_002E": "year_structure_built_2014_or_later",
                                      "B25034_003E": "year_structure_built_2010_to_2013",
                                      "B25034_004E": "year_structure_built_2000_to_2009",
                                      "B25034_005E": "year_structure_built_1990_to_1999",
                                      "B25034_006E": "year_structure_built_1980_to_1989",
                                      "B25034_007E": "year_structure_built_1970_to_1979",
                                      "B25034_008E": "year_structure_built_1960_to_1969",
                                      "B25034_009E": "year_structure_built_1950_to_1959",
                                      "B25034_010E": "year_structure_built_1940_to_1949",
                                      "B25034_011E": "year_structure_built_1939_or_earlier",
                                      "NAME": "name", "zip code tabulation area": "zipcode"})

In [6]:
# Add in poverty rate (poverty count / population)
census_pd["poverty_rate"] = 100 * (census_pd["poverty_count"] / census_pd["population"])

In [7]:
# Final DataFrame

census2018_pd = census_pd[["zipcode", "population", "median_age", "median_household_income", "per_capita_income",
                         "poverty_count", "poverty_rate", "year_structure_built_1939_or_earlier",
                         "year_structure_built_1940_to_1949", "year_structure_built_1950_to_1959",
                         "year_structure_built_1960_to_1969", "year_structure_built_1970_to_1979",
                         "year_structure_built_1980_to_1989", "year_structure_built_1990_to_1999",
                         "year_structure_built_2000_to_2009", "year_structure_built_2010_to_2013",
                         "year_structure_built_2014_or_later", "year_structure_built_total"]]

In [8]:
uszipcode5_file = "uszipcodes.csv"
df_uszipcode5 = pd.read_csv(uszipcode5_file)

In [9]:
uszipcode4_file = "uszipcodes4.csv"
df_uszipcode4 = pd.read_csv(uszipcode4_file)

In [10]:
uszipcode3_file = "uszipcodes3.csv"
df_uszipcode3 = pd.read_csv(uszipcode3_file)

In [11]:
## Add leading zeros to the integer column in Python

df_uszipcode3['zipcode']=df_uszipcode3['zipcode'].apply(lambda x: '{0:0>5}'.format(x))

In [12]:
## Add leading zeros to the integer column in Python

df_uszipcode4['zipcode']=df_uszipcode4['zipcode'].apply(lambda x: '{0:0>5}'.format(x))

In [13]:
df_uszipcode5['zipcode']=df_uszipcode5['zipcode'].apply(lambda x: '{0:0>5}'.format(x))

In [14]:
tempzip = [df_uszipcode3, df_uszipcode4, df_uszipcode5]
df_uszipcode = pd.concat(tempzip)

In [15]:
census_combined_pd = pd.merge(census2018_pd, df_uszipcode, on = "zipcode", how = "outer")

In [16]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_combined_pd.to_csv("census_data_2018.csv", encoding="utf-8", index=False)